In [ ]:
WHATTUPP

hi guys

In [3]:
library(tidyverse)
library(repr)
library(tidymodels)
library(janitor)
options(repr.matrix.max.rows = 6)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
url_players <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
url_sessions <- "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"

player_data <- read_csv(url_players) |>
    clean_names()
player_data 



Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashed_email,played_hours,name,gender,age,individual_id,organization_name
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17,NA,NA
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,NA,NA
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,91,NA,NA


In [6]:
players <- player_data |>
    mutate(Pro = ifelse(experience == "Pro", 1, 0),
           Amateur = ifelse(experience == "Amateur", 1, 0),
           Veteran = ifelse(experience == "Veteran", 1, 0),
           Beginner = ifelse(experience == "Beginner", 1, 0),
           Regular = ifelse(experience == "Regular", 1, 0)) |>
    select(-individual_id, -hashed_email, -organization_name)
players





experience,subscribe,played_hours,name,gender,age,Pro,Amateur,Veteran,Beginner,Regular
<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pro,TRUE,30.3,Morgan,Male,9,1,0,0,0,0
Veteran,TRUE,3.8,Christian,Male,17,0,0,1,0,0
Veteran,FALSE,0.0,Blake,Male,17,0,0,1,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,0.0,Dylan,Prefer not to say,17,0,1,0,0,0
Amateur,FALSE,2.3,Harlow,Male,17,0,1,0,0,0
Pro,TRUE,0.2,Ahmed,Other,91,1,0,0,0,0


In [7]:
player_split <- initial_split(players, prop = 0.75, strata = played_hours)
player_training <- training(player_split)
player_training
player_testing <- testing(player_split)
player_testing

experience,subscribe,played_hours,name,gender,age,Pro,Amateur,Veteran,Beginner,Regular
<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Veteran,FALSE,0.0,Blake,Male,17,0,0,1,0,0
Regular,TRUE,0.1,Kylie,Male,21,0,0,0,0,1
Amateur,TRUE,0.0,Adrian,Female,17,0,1,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,TRUE,0.8,Hunter,Female,22,0,1,0,0,0
Amateur,TRUE,32.0,Sidney,Male,22,0,1,0,0,0
Amateur,FALSE,2.3,Harlow,Male,17,0,1,0,0,0


experience,subscribe,played_hours,name,gender,age,Pro,Amateur,Veteran,Beginner,Regular
<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Amateur,FALSE,0.0,Emerson,Male,21,0,1,0,0,0
Beginner,TRUE,1.5,Sarah,Female,25,0,0,0,1,0
Amateur,TRUE,0.5,Marley,Male,17,0,1,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Veteran,TRUE,0.1,Gabriela,Female,44,0,0,1,0,0
Amateur,TRUE,0.0,Rhys,Male,20,0,1,0,0,0
Amateur,TRUE,0.0,Bailey,Female,17,0,1,0,0,0


In [11]:
player_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
      set_engine("kknn") |>
      set_mode("regression") 

player_recipe <- recipe(played_hours ~ age + Pro + Amateur + Veteran + Beginner + Regular, data = players ) |>
      step_scale(all_predictors()) |>
      step_center(all_predictors())
player_recipe




── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 6



── Operations 

• Scaling for: all_predictors()

• Centering for: all_predictors()



In [12]:

player_vfold <- vfold_cv(player_training, v = 5, strata = played_hours)

player_workflow <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(player_spec)
player_workflow

gridvals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

player_results <- player_workflow |>
  tune_grid(resamples = player_vfold, grid = gridvals) |>
  collect_metrics()
player_results

══ Workflow ════════════════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
2 Recipe Steps

• step_scale()
• step_center()

── Model ───────────────────────────────────────────────────────────────────────
K-Nearest Neighbor Model Specification (regression)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,rmse,standard,39.45353291,5,12.07302462,Preprocessor1_Model01
1,rsq,standard,0.02880266,5,0.02477363,Preprocessor1_Model01
2,rmse,standard,22.05497891,5,5.90217324,Preprocessor1_Model02
⋮,⋮,⋮,⋮,⋮,⋮,⋮
9,rsq,standard,0.01699750,5,0.010133494,Preprocessor1_Model09
10,rmse,standard,21.07948198,5,6.309310468,Preprocessor1_Model10
10,rsq,standard,0.01761082,5,0.009540272,Preprocessor1_Model10


In [ ]:
 player_min <- player_results |>
#    filter(.metric == "rmse") |>
#    slice_min(std_err, n = 1)
# player_min

# k_min <- player_min |>
#          pull(neighbors)

# player_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) |>
#          set_engine("kknn") |>
#          set_mode("regression")

# player_best_fit <- workflow() |>
#          add_recipe(player_recipe) |>
#          add_model(player_best_spec) |>
#          fit(data = player_training)

In [ ]:
player_summary <- player_best_fit |>
#           predict(player_testing) |>
#           bind_cols(player_testing) |>
#           metrics(truth = played_hours, estimate = .pred) 

# player_preds <- player_best_fit |>
#   predict(player_training) |>
#   bind_cols(player_training)

# player_plot <- player_preds |>
# ggplot(aes(x = max, y = played_hours)) + 
# geom_point(alpha = 0.4) +
# geom_line(data = player_preds,
#           mapping = aes(x = max, y = .pred), 
#           color = "blue",) +
# labs(x = "Factors", 
#      y = "Number of Hours Played") +
# ggtitle("Factors that Influence the Number of Hours Played") +
# theme(text = element_text(size = 15))


# player_plot <- 
